In [34]:


import numpy as np #thư viện
import pandas as pd  #thư viện 
import seaborn as sns # thư viện vẽ biểu đồ
import matplotlib.pyplot as plt  #  thư viện vẽ đồ thị dựa trên số liệu
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, auc, roc_curve
import graphviz
import warnings
warnings.filterwarnings('ignore')
#test
from sklearn.pipeline import Pipeline


   # nhập dữ liệu
wine = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
wine.head(9)



In [4]:
list(wine.columns) # show ra 12 thuộc tính trong dataset

In [5]:
#show ra số lượng trong dataset

In [6]:
wine.shape

In [7]:
wine.dtypes #  show ra kiểu dữ liệu các thuộc tính

In [8]:
wine.info() #show thông tin 

In [9]:
wine.isnull().sum()   #kiem tra có dữ liệu nào không có giá trị 

In [10]:
wine.describe()   #mô tả dữ liệu

In [11]:
wine["quality"].value_counts()    #Đếm số lượng của từng chất lượng rượu từ 3-8

In [12]:
sns.set()
sns.catplot(x='quality', data=wine, kind='count', height=9)    #vẽ biểu đồ của thuộc tính chất lượng

In [13]:
correlations = wine.corr()["quality"].sort_values(ascending=False)
correlations

# so sánh  mức độ tương quan của chất lượng rượu với các thuộc tính khác

In [14]:
correlations.plot(kind="bar")  # vẽ đồ thị 

In [15]:
plt.figure(figsize=(12,8))
sns.heatmap(wine.corr(),annot=True,cmap="coolwarm")  
# đây là  biểu đồ để hiểu rõ các đối tượng tương quan

In [16]:
sns.pairplot(wine)

In [17]:
# separate the data and Label
X = wine.drop('quality',axis=1)

In [18]:
X

In [19]:
Y= wine['quality'].apply(lambda y: 1 if y>=7 else 0)
Y.head(20)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
print(wine.shape)
print(X.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [22]:


from sklearn.tree import DecisionTreeClassifier as clf
Y = ['good','bad']
clf = DecisionTreeClassifier(max_depth=4,random_state=42)
clf.fit(X_train, Y_train)
Y_pred_dt = clf.predict(X_test)
dot_data = export_graphviz(clf, out_file=None, 
                         feature_names=X.columns,
                         class_names=Y, 
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)
#graph.render(filename='DecisionTree')
graph

In [23]:
#Visualize biểu đồ thể hiện các đặc tính quan trọng trong bộ dữ liệu 
features_list = X.columns.values
feature_importance = clf.feature_importances_
sorted_idx = np.argsort(feature_importance)

plt.figure(figsize=(10,5))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), features_list[sorted_idx])
plt.xlabel('Importance')
plt.title('Feature importances')
plt.draw()
#plt.savefig("featureimp.png", format='png', dpi=100, bbox_inches='tight')
plt.show()

In [24]:
from sklearn.model_selection import cross_val_predict, cross_val_score

print("TEST RESULTS:\n")
#Report
print('Decision Tree Classifier Report:\n\n{}\n'.format(classification_report(Y_test, Y_pred_dt)))
res = cross_val_score(clf, X_test,Y_test, cv=10, n_jobs=-1, scoring='accuracy')
#Độ chính xác trung bình
print('Average Accuracy:\t{0:.4f}\n'.format((res.mean())))
#Độ lệch chuẩn
print('Standard Deviation:\t{0:.4f}\n'.format(res.std()))
#Ma trận hỗn hợp
print('Confusion Matrix:\n{}\n'.format(confusion_matrix(Y_test,clf.predict(X_test))))
#Điểm chính xác
print("Accuracy Score:\t\t{}%".format(round(clf.score(X_test, Y_test)*100, 2)))

In [40]:
#red wine khai báo thư viện knn, 
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)


In [41]:
#bao nhiu k là ít lỗi nhất
error_rate = []
for i in range(1, 40):
	
	knn = KNeighborsClassifier(n_neighbors = i)
	knn.fit(X_train, Y_train)
	pred_i = knn.predict(X_test)
	error_rate.append(np.mean(pred_i != Y_test))

plt.figure(figsize =(10, 6))
plt.plot(range(1, 40), error_rate, color ='blue',
				linestyle ='dashed', marker ='o',
		markerfacecolor ='red', markersize = 10)

plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [28]:
#K=4 has lowest error rate
#Model Fit
classifier2 = KNeighborsClassifier(n_neighbors= 4, metric = 'manhattan', p = 2,weights='uniform')
classifier2.fit(X_train,Y_train)

In [30]:
# Predicting the ouput from input data (x_train) and (y_train) 
y_pred1 = classifier2.predict(X_train)
y_pred2 = classifier2.predict(X_test)

In [42]:
from sklearn.metrics import accuracy_score
print("Độ chính xác của tập huấn luyện:",accuracy_score(Y_train, y_pred1))
print("Độ chính xác của tập kiểm thử :",accuracy_score(Y_test, y_pred2))

In [43]:
plt.figure()
plt.plot(Y_test,'o',color = 'blue',label = 'Giá trị dự đoán ') #dự đoán
plt.plot(y_pred2,color = 'red',label = 'Giá trị thực tế') # thực tế
plt.legend()

In [46]:
from sklearn.preprocessing import MinMaxScaler
k = range(1,50,2)
testing_accuracy = []
training_accuracy = []
score = 0
#Fitting the model
for i in k:
    knn = KNeighborsClassifier(n_neighbors = i)
    pipe_knn = Pipeline([('scale', MinMaxScaler()), ('knn', knn)])
    pipe_knn.fit(X_train, Y_train)
    
    y_pred_train = pipe_knn.predict(X_train)
    training_accuracy.append(accuracy_score(Y_train, y_pred_train))
    
    y_pred_test = pipe_knn.predict(X_test)
    acc_score = accuracy_score(Y_test,y_pred_test)
    testing_accuracy.append(acc_score)
    
    if score < acc_score:
        score = acc_score
        best_k = i
        
print('Độ chính xác tốt nhất ', score, ' ,K tốt nhất', best_k)

In [48]:
from sklearn.model_selection import cross_val_predict, cross_val_score

print("Kết quả test:\n")
#Report
print('K-Nearest Neighbors Báo cáo:\n\n{}\n'.format(classification_report(Y_test, y_pred_test)))
res = cross_val_score(knn, X_test, Y_test, cv=10, n_jobs=-1, scoring='accuracy')
#Độ chính xác trung bình
print('Độ chính xác trung bình :\t{0:.4f}\n'.format((res.mean())))
#Độ lệch chuẩn
print('Độ lệch chuẩn:\t{0:.4f}\n'.format(res.std()))
#Ma trận hỗn hợp
print('Ma trận hỗn hợp:\n{}\n'.format(confusion_matrix(Y_test,knn.predict(X_test))))
#Điểm chính xác
print("Điểm chính xác:\t\t{}%".format(score*100, 2))

In [50]:
con_mat = confusion_matrix(Y_test,y_pred_test)


print('\nCONFUSION MATRIX')
plt.figure(figsize= (6,4))
sns.heatmap(confusion_matrix(Y_test,knn.predict(X_test)), annot = True,fmt='d',cmap="YlGnBu")